# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [26]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [27]:
# TODO: Import the necessary libs
# For example: 
import os, json, re
from dotenv import load_dotenv
from typing import List, Dict, Annotated
from pydantic import BaseModel,Field
from datetime import datetime
import pandas as pd

from tavily import TavilyClient

from rich.console import Console
from rich.markdown import Markdown
console=Console()

from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage, BaseMessage
from lib.tooling import tool
from lib.vectordb import VectorStore

import chromadb
from chromadb.api.types import QueryResult

In [28]:
# TODO: Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [29]:
# TODO: Create retrieve_game tool
# It should use chroma client and collection you created
# chroma_client = chromadb.PersistentClient(path="chromadb")
# collection = chroma_client.get_collection("udaplay")
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - query: a question about game industry. 
#
#    You'll receive results as list. Each element contains:
#    - Platform: like Game Boy, Playstation 5, Xbox 360...)
#    - Name: Name of the Game
#    - YearOfRelease: Year when that game was released for that platform
#    - Description: Additional details about the game
class retrievedDocument(BaseModel):
    """Pydantic data model for retrieved documents from the vectorstore"""
    Platform: Annotated[str, Field(description="Game platform (PC, Xbox, Wii...)")]
    Name: Annotated[str, Field(description="Title of the game")]
    YearOfRelease: Annotated[int, Field(description="Year of release")]
    Description: Annotated[str, Field(description="One-liner game description")]


@tool
def retrieve_game(query:str, num_matches:int=5)->List[retrievedDocument]:
    """
    Semantic search: Finds N results in the vector DB closest to the query
    args:
    - query: a question about game industry. 
    - num_matches (int): Number of matches to return (default is 5) - Use this parameter to increase the number of matches returned      

    You'll receive results as list up to num_matches length. Each element contains:
    - Platform: like Game Boy, Playstation 5, Xbox 360...)
    - Name: Name of the Game
    - YearOfRelease: Year when that game was released for that platform
    - Description: Additional details about the game
    """
    chroma_client = chromadb.PersistentClient(path="chromadb")
    collection = chroma_client.get_collection("udaplay")
    num_records = collection.count()
    vectorstore_db = VectorStore(chroma_collection=collection)
    outputs:QueryResult = vectorstore_db.query(
        query_texts=[query],
        n_results=num_matches,          # collect the maximum results   - TODO CHECK LOGIC BECAUSE LARGE CONTEXT
        where_document=None,
        where=None
        )

    if outputs:
        metadatas = outputs['metadatas'][0]
        platforms=[];names=[];years=[];descriptions=[]
        # Use pydantic data model
        # Now you can access them as model objects, ie result.Name, result.Description, ect...
        documents = [retrievedDocument(**metadata) for metadata in metadatas]
        
    return documents


@tool
def retrieve_all_games()->List[retrievedDocument]:
    """
    Retrieves all records from the database
    Output:
    You'll receive all records as list. Each element contains:
    - Platform: like Game Boy, Playstation 5, Xbox 360...)
    - Name: Name of the Game
    - YearOfRelease: Year when that game was released for that platform
    - Description: Additional details about the game
    """
    chroma_client = chromadb.PersistentClient(path="chromadb")
    collection = chroma_client.get_collection("udaplay")
    # Retrieve all records
    all_records = collection.get()

    if all_records:
        metadatas = all_records['metadatas']
        platforms=[];names=[];years=[];descriptions=[]
        # Use pydantic data model
        # Now you can access them as model objects, ie result.Name, result.Description, ect...
        documents = [retrievedDocument(**metadata) for metadata in metadatas]
        
    return documents  

#### Evaluate Retrieval Tool

In [30]:
# TODO: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result
# Tool Docstring:
#    Based on the user's question and on the list of retrieved documents, 
#    it will analyze the usability of the documents to respond to that question. 
#    args: 
#    - question: original question from user
#    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
#    The result includes:
#    - useful: whether the documents are useful to answer the question
#    - description: description about the evaluation result

all_records = retrieve_all_games()
console.print(f"{len(all_records)} retrieved from the database. Examples: ", style='bold yellow')
all_records[:3]

48 retrieved from the database. Examples: 

[retrievedDocument(Platform='PlayStation 1', Name='Gran Turismo', YearOfRelease=1997, Description='A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.'),
 retrievedDocument(Platform='PlayStation 2', Name='Grand Theft Auto: San Andreas', YearOfRelease=2004, Description="An expansive open-world game set in the fictional state of San Andreas, following the story of Carl 'CJ' Johnson."),
 retrievedDocument(Platform='PlayStation 3', Name='Gran Turismo 5', YearOfRelease=2010, Description='A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics.')]

In [31]:
temp={}
for i in range(len(all_records)):
    temp[i] = pd.DataFrame(json.loads(all_records[i].model_dump_json()), index=[i])
records_df = pd.concat(temp,ignore_index=True)
records_df

,Platform,Name,YearOfRelease,Description
0,PlayStation 1,Gran Turismo,1997,A realistic racing simulator featuring a wide ...
1,PlayStation 2,Grand Theft Auto: San Andreas,2004,An expansive open-world game set in the fictio...
2,PlayStation 3,Gran Turismo 5,2010,A comprehensive racing simulator featuring a v...
3,PlayStation 4,Marvel's Spider-Man,2018,An open-world superhero game that lets players...
4,PlayStation 5,Marvel's Spider-Man 2,2023,"The sequel to the acclaimed Spider-Man game, f..."
5,Game Boy Color,Pokémon Gold and Silver,1999,Second-generation Pokémon games introducing ne...
6,Game Boy Advance,Pokémon Ruby and Sapphire,2002,Third-generation Pokémon games set in the Hoen...
7,Super Nintendo Entertainment System (SNES),Super Mario World,1990,A classic platformer where Mario embarks on a ...
8,Nintendo 64,Super Mario 64,1996,A groundbreaking 3D platformer that set new st...
9,GameCube,Super Smash Bros. Melee,2001,A crossover fighting game featuring characters...


In [32]:
table = pd.pivot_table(
    records_df, 
    index=['Platform'], 
    columns=['YearOfRelease'], 
    values=['Name'],
    aggfunc='count',
    fill_value=0,
    margins=True, 
    margins_name="Total" 
    )

table.replace(0,'')

Name                                \
YearOfRelease                              1990 1996 1997 1999 2001 2002 2004   
Platform                                                                        
Game Boy Advance                                                       1        
Game Boy Color                                               1                  
GameCube                                                          1             
Nintendo 64                                        1                            
Nintendo Switch                                                                 
PC                                                                              
PlayStation 1                                           1                       
PlayStation 2                                                               1   
PlayStation 3                                                                   
PlayStation 4                                                                   
PlayStation 5                                                                   
Super Nintendo Entertainment System (SNES)    1                                 
Wii                                                                             
Windows                                                                         
Xbox 360                                                                        
Xbox One                                                                        
Xbox Series X/S                                                                 
Xbox Series X|S                                                                 
Total                                         1    1    1    1    1    1    1   

                                                                               \
YearOfRelease                              2006 2010 2014 2017 2018 2021 2022   
Platform                                                                        
Game Boy Advance                                                                
Game Boy Color                                                                  
GameCube                                                                        
Nintendo 64                                                                     
Nintendo Switch                                              1                  
PC                                                                              
PlayStation 1                                                                   
PlayStation 2                                                                   
PlayStation 3                                      1                            
PlayStation 4                                                     1         1   
PlayStation 5                                                               1   
Super Nintendo Entertainment System (SNES)                                      
Wii                                           1                                 
Windows                                                                         
Xbox 360                                           1                            
Xbox One                                                1                       
Xbox Series X/S                                                        1    1   
Xbox Series X|S                                                        1        
Total                                         1    2    1    1    1    2    3   

                                                                 
YearOfRelease                              2023 2024 2025 Total  
Platform                                                         
Game Boy Advance                                              1  
Game Boy Color                                                1  
GameCube                                                      1  
Nintendo 64                                                   1  
Nintendo Switch

Evaluate Retrieval Tool

In [50]:
class evaluationResult(BaseModel):
    """pydantic data model for evaluation result"""
    useful: Annotated[bool, Field(description="whether the documents are useful to answer the question")]
    description: Annotated[str, Field(description="description about the evaluation result")]

@tool
def evaluate_retrieval(question:str, retrieved_docs:List[retrievedDocument])->evaluationResult:
    """
    Based on the user's question and on the list of retrieved documents, 
    it will analyze the usability of the documents to respond to that question. 
    args: 
    - question: original question from user
    - retrieved_docs: Full context of documents available to answer the question
    The result includes:
    - useful: whether the documents are useful to answer the question
    - description: description about the evaluation result
    """

    llm = LLM(
        api_key=OPENAI_API_KEY, 
        model='gpt-4o', 
        temperature=0
        )
    
    system_prompt = ("You are an expert at evaluating other LLM's responses.\n"
    "Your task is to evaluate whether the documents retrieved are useful to answer the question from the user. "
    "Ask yourself whether you can answer the user question based on the information provided."
    )
    
    user_prompt= ("Evaluate this output: "
    f"user question: {question}\n"
    f"retrieved documents:\n{retrieved_docs}"
    )

    messages=[
        SystemMessage(content=system_prompt),
        UserMessage(content=user_prompt)
    ]

    response = llm.invoke(
        input=messages,  
        )

    return response.content   

#### Game Web Search Tool

In [51]:
# TODO: Create game_web_search tool
# Please use Tavily client to search the web
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - question: a question about game industry. 
@tool
def web_search(query: str, search_depth: str = "advanced") -> Dict:
    """
    Search the web using Tavily API
    args:
        query (str): Search query
        search_depth (str): Type of search - 'basic' or 'advanced' (default: advanced)
    """
    api_key = os.getenv("TAVILY_API_KEY")
    client = TavilyClient(api_key=api_key)
    
    # Perform the search
    search_result = client.search(
        query=query,
        search_depth=search_depth,
        include_answer=True,
        include_raw_content=False,
        include_images=False
    )
    
    # Format the results
    formatted_results = {
        "answer": search_result.get("answer", ""),
        "results": search_result.get("results", []),
        "search_metadata": {
            "timestamp": datetime.now().isoformat(),
            "query": query
        }
    }
    
    return formatted_results

### Agent

In [57]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed

tools = [retrieve_game,retrieve_all_games,evaluate_retrieval, web_search]

instructions = (
    "You are an helpful assistant with deep knowledge on computer games. "
    "You answer user request and provide faithful, detailed and structured information. "
    "You first use semantic search in your VectorDB to answer questions on computer games. "
    "Always reason about the response, rephrase the question and call the tool again until you get a satisfiying result. "
    "In case the information is not present, you can use the web search tool. "
    "You always check the information with the evaluate_retrieval tool before answering the question. Make sure to run the evaluation tool just before your final response. "
    "If you do not know the answer just answer 'I dont know'. "
    "Be concise. Answer with bullet points citing your sources. "
    f"Note: your database contains {chromadb.PersistentClient(path="chromadb").get_collection("udaplay").count()} records on the most successful games"
)

agent = Agent(
    model_name="gpt-4o",
    tools=tools,
    instructions=instructions
)

In [58]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?
# placeholders for color priting
ENDC = '\33[0m'
RED = '\33[31m'
GREEN = '\33[32m'
YELLOW = '\33[33m'
VIOLET = '\33[35m'
CYAN = '\33[36m'
def print_messages(messages: List):
    """
    pretty print a list of AI messages with color-code using rich and ANSI
    inputs:
        - list of AI messages
    """
    for m in messages:

        content = m.content

        # If this is a tool response containing retrievedDocument objects, pretty-print them - No color
        if isinstance(content, str) and "retrievedDocument(" in content:
            docs = re.findall(r"retrievedDocument\([^)]+\)", content)
            if docs:
                print(f" -> role = {m.role}, tool_calls = {getattr(m, 'tool_calls', None)}")
                print("    content =")
                for doc in docs:
                    print(f"       {doc}")
            else:
                print(f" -> role = {m.role}, content = {content}, tool_calls = {getattr(m, 'tool_calls', None)}")

        # if this is another tool response, pretty print tool call result - No color
        elif isinstance(content, str) and m.role=="tool" :
            print(f" -> role = {m.role}, content = {json.loads(content)}, tool_calls = {getattr(m, 'tool_calls', None)}")

        # if this is a tool calling, print in CYAN color
        elif m.role=="assistant" and m.tool_calls:
            print(CYAN, f" -> role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)}", ENDC)

        # if this is a system prompt, print in VIOLET color
        elif m.role=="system" :
            print(VIOLET, f" -> role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)}", ENDC)

        # if this is the initial user prompt, print in GREEN color
        elif m.role=="user":
            print(GREEN, f" -> role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)}", ENDC)

        # print in YELLOW all generated response by the assistant
        else:
            print(YELLOW, f" -> role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)}",ENDC)

        print('-----------------------------------------------------------')

    # Final message is the assistant's last content - Print final resposne in GREEN color
    if messages:
        last_msg = messages[-1]
        console.print('Final Answer:', style='bold green')
        console.print(Markdown(last_msg.content), style='bold green')

### (Optional) Advanced

In [59]:
def process_question(question:str, session_id:str=None)->None:
    """
    LLM agent answers question in a specified session
    Workflow messages are printed in addition to final response
    inputs:
        - question from the user (str)
        - session_id (Optional[str])

    outputs:
        - full list of messages from the session_id
    """
    response = agent.invoke(
        query=question,
        session_id=session_id
        )

    print(f"\nMessages from {session_id or 'default'}:")
    messages = response.get_final_state()["messages"]
    print_messages(messages)

    return messages

In [60]:
query = "When Pokémon Gold and Silver was released?"
response = agent.invoke(
    query=query,
    session_id='question_0'
    )

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}